In [1]:
import sys
print(sys.executable)

/home/uzan/GlobalEduAgent/edu-finetuning/bin/python


In [2]:
import torch
print("PyTorch 版本:", torch.__version__)
print(torch.cuda.is_available())
print("PyTorch 使用的 CUDA 版本:", torch.version.cuda)

PyTorch 版本: 2.4.1+cu121
True
PyTorch 使用的 CUDA 版本: 12.1


In [3]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/uzan/GlobalEduAgent/edu-finetuning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
max_seq_length=4096
dtype=None
load_in_4bit=True

model_name="unsloth/tinyllama-bnb-4bit"

model, tokenizer= FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/tinyllama-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!


In [34]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules = ["q_proj","k_proj","v_proj","o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias= "none",
    use_gradient_checkpointing=False,
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

model.config.dropout = 0.2        # 增加原始dropout概率
model.config.attention_dropout = 0.1

Unsloth: Already have LoRA adapters! We shall skip this step.


In [10]:
prompt_template="""
[INST]
<<SYS>>
你是一名资深的留学顾问。用分布推理的方式回答客户的问题，先分析条件，再给出建议。
<</SYS>>
### Question:
{}
[/INST]
### Answer:
{}
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    questions       = examples["question"]
    answers      = examples["answer"]
    texts = []
    for question, answer in zip(questions, answers):
        text = prompt_template.format(question.strip(), answer.strip()) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }



In [11]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="edu_training_data.json",split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 569.46 examples/s]


In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [39]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Packs short sequences together to save time!
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 200,
        learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [29]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4060 Laptop GPU. Max memory = 7.996 GB.
1.449 GB of memory reserved.


In [47]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 200 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 2,252,800/617,859,072 (0.36% trained)


Step,Training Loss
1,0.938100
2,0.938100
3,0.937400
4,0.935000
5,0.934900
6,0.939000
7,0.933400
8,0.936700
9,0.934300
10,0.931500


In [41]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt_template.format(
        "我能申请X大学吗", #question
        ""
    )
],
return_tensors="pt").to("cuda")

In [48]:
outputs=model.generate(**inputs,max_new_tokens=64, use_cache=True)
tokenizer.batch_decode(outputs)

['<s> \n[INST]\n<<SYS>>\n你是一名资深的留学顾问。用分布推理的方式回答客户的问题，先分析条件，再给出建议。\n<</SYS>>\n### Question:\n我能申请X大学吗\n[/INST]\n### Answer:\n\n*\n*\nX大学需要顾问账单总金额几百万美元。可以使用本地方留学服务的方式申资。\n<</SYS>>\n###']